In [ ]:
python train.py --data colon --snapshot_path "snapshot" --data_prefix "dataset/Task10_Colon" 

In [6]:
import cv2
import numpy as np
import nibabel as nib
from scipy import ndimage

# 讀取nii.gz檔案
img = nib.load('3DSAM-adapter\dataset\Task10_Colon\imagesTr\colon_001.nii.gz')
gt = nib.load('3DSAM-adapter\dataset\Task10_Colon\labelsTr\colon_001.nii.gz')

# 獲取metadata的解析度
zooms = img.header.get_zooms()

# 內插3D影像回(1,1,1)的解析度
print(img.get_fdata().shape, gt.get_fdata().shape)
rescaled_img = ndimage.zoom(img.get_fdata(), zooms)
rescaled_gt = ndimage.zoom(gt.get_fdata(), zooms)
print(rescaled_img.shape, rescaled_gt.shape)

# 根據每張圖像的最小值和最大值進行正規化
min_val, max_val = np.min(rescaled_img), np.max(rescaled_img)
rescaled_img = np.interp(rescaled_img, (min_val, max_val), (0, 255)).astype(np.uint8)

# 將內插後的ground truth轉換成0或1的二元圖
rescaled_gt = np.where(rescaled_gt > 0.5, 255, 0).astype(np.uint8)

# 將內插完的圖像保存下來
np.save('rescaled_img.npy', rescaled_img)
np.save('rescaled_gt.npy', rescaled_gt)

(512, 512, 60) (512, 512, 60)
(0.779297, 0.779297, 7.5)


In [7]:
# 載入保存的圖像
rescaled_img = np.load('rescaled_img.npy')
rescaled_gt = np.load('rescaled_gt.npy')

# 創建一個bar來改變顯示的slice
max_value = rescaled_img.shape[2] - 1  # 最大值為z軸的slice數量減一
bar_name = 'Slice number'
cv2.namedWindow('CT and Ground Truth', cv2.WINDOW_NORMAL)
cv2.createTrackbar(bar_name, 'CT and Ground Truth', 0, max_value, lambda x: None)

while True:
    # 獲取當前bar的值，並顯示對應的slice
    slice_num = cv2.getTrackbarPos(bar_name, 'CT and Ground Truth')
    ct_slice = rescaled_img[:, :, slice_num]
    gt_slice = rescaled_gt[:, :, slice_num]

    # 將影像逆時針旋轉90度
    ct_slice = np.rot90(ct_slice)
    gt_slice = np.rot90(gt_slice)

    # 將CT image和ground truth並排顯示
    combined_img = np.hstack((ct_slice, gt_slice))

    # 顯示圖片
    cv2.imshow('CT and Ground Truth', combined_img)

    # 按下'q'鍵退出迴圈
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cv2.destroyAllWindows()


error: OpenCV(4.8.1) D:\a\opencv-python\opencv-python\opencv\modules\highgui\src\window_w32.cpp:2561: error: (-27:Null pointer) NULL window: 'CT and Ground Truth' in function 'cvGetTrackbarPos'


In [ ]:
import nibabel as nib
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import os
import fnmatch
from typing import Union
from scipy.ndimage import zoom
import vtkplotter


class DataViewer3D:
    def __init__(self, idx=None, img_path=None, label_path=None) -> None:
        assert img_path or label_path
        self.img_path = img_path
        self.label_path = label_path
        self.idx = idx
        self.imgs = self.load_data(self.get_filelist(self.img_path),self.idx)
        self.labels = self.load_data(self.get_filelist(self.label_path),self.idx)

    def get_filelist(self, dir) -> list[str]:
        if dir==None:
            return None
        matches = list()
        for root, dirnames, filenames in os.walk(dir):
            for filename in fnmatch.filter(filenames, "*.nii.gz"):
                matches.append(os.path.join(root, filename))
        return matches
    
    def load_data(self, filelist, idx=None) -> Union[list[np.ndarray], np.ndarray]:
        if idx is not None:
            data_nii = nib.load(filelist[idx])
            data_np = data_nii.get_fdata()
            
            volume = vtkplotter.load(filelist[idx]) #returns a vtkVolume object
            vtkplotter.show(volume, bg='white')
            return data_np
        else:
            datas = list()
            for f_name in filelist:
                data_nii = nib.load(f_name)
                data_np = data_nii.get_fdata()
                datas.append(data_np)
            return datas
        
    
    def plot_data(self):
        assert len(self.imgs) == len(self.labels)
        if self.idx is not None:
            pass
        else:
            for idx in range(len(self.imgs)):
                fig = plt.figure()

                # 繪製醫學影像
                ax1 = fig.add_subplot(121, projection='3d')
                ax1.title.set_text('Medical Image')
                ax1.voxels(self.imgs[idx])

                # 繪製ground truth
                ax2 = fig.add_subplot(122, projection='3d')
                ax2.title.set_text('Ground Truth')
                ax2.voxels(self.labels[idx])

                plt.show()

dv = DataViewer3D(idx = 0, img_path = "3DSAM-adapter\dataset\Task10_Colon\imagesTr", label_path = "3DSAM-adapter\dataset\Task10_Colon\labelsTr")
dv.plot_data()

In [42]:
import os
import numpy as np
import nibabel as nib
from scipy import ndimage
from tqdm import tqdm

def resample_label(image_path):
    img = nib.load(image_path)
    zooms = img.header.get_zooms()
    rescaled_img = ndimage.zoom(img.get_fdata(), zooms, order=0) # 使用最近鄰插值（nearest-neighbor interpolation)

    # 四捨五入label
    rescaled_img = np.round(rescaled_img).astype(np.uint8)
    return rescaled_img

def seperate_label(data, num_class):
    separated_arrays = []
    for i in range(1, num_class + 1):
        separated_array = np.where(data == i, i, 0)
        separated_arrays.append(separated_array)
    return separated_arrays

def find_extreme_coords(data):
    non_zero_indices = np.nonzero(data)

    if non_zero_indices[0].size == 0: # 如果是全0矩陣
        return np.array([0,0,0]), np.array([0,0,0])

    min_coords = np.min(non_zero_indices, axis=1)
    max_coords = np.max(non_zero_indices, axis=1)
    return min_coords, max_coords


def get_sizes_of_labels(label_folder, num_class, txt_file):
    x = [[] for _ in range(num_class)]
    y = [[] for _ in range(num_class)]
    z = [[] for _ in range(num_class)]
    for label_file in tqdm(os.listdir(label_folder)):
        if label_file.endswith('.nii.gz'):
            # load and interpolate label
            data = resample_label(os.path.join(label_folder, label_file))

            # separate multiple lables
            array_list = seperate_label(data, num_class)

            assert len(array_list) == num_class, f"{label_file}: array_length{len(array_list)}, num_class{num_class}不一致"

            # get min and max coordinates
            for i, array in enumerate(array_list):
                min_coords, max_coords = find_extreme_coords(array)
                length = max_coords - min_coords

                x[i].append(length[0])
                y[i].append(length[1])
                z[i].append(length[2])
                if length[0]==0 and length[1] == 0 and length[2] == 0:
                    txt_file.write(f"{label_file} {i}th label empty\n")
    return np.array(x), np.array(y), np.array(z)

base_dir = "D:\SAM"
folder_names = ["Task01_BrainTumour", "Task02_Heart", "Task03_Liver","Task04_Hippocampus","Task05_Prostate","Task06_Lung","Task07_Pancreas","Task08_HepaticVessel","Task09_Spleen","Task10_Colon"]
num_classes = [3,1,2,2,2,1,2,2,1,1]


for task_num, f in enumerate(folder_names):
    print(f"{f}, {num_classes[task_num]}類")
    with open('data_empty.txt', 'a') as txt_file:
        # x,y,z = get_sizes_of_labels(os.path.join("3DSAM-adapter\dataset\Task10_Colon", "labelsTr"), num_classes[task_num])
        x,y,z = get_sizes_of_labels(os.path.join(base_dir, f, f, "labelsTr"), num_classes[task_num], txt_file)
        print("寬度:", x.shape)
        print("高度:", y.shape)
        print("深度:", z.shape)
    np.save(f+"_x.npy", x)
    np.save(f+"_y.npy", y)
    np.save(f+"_z.npy", z)

Task01_BrainTumour, 3類


100%|██████████| 484/484 [02:09<00:00,  3.73it/s]


寬度: (3, 484)
高度: (3, 484)
深度: (3, 484)
Task02_Heart, 1類


100%|██████████| 20/20 [00:13<00:00,  1.48it/s]


寬度: (1, 20)
高度: (1, 20)
深度: (1, 20)
Task03_Liver, 2類


...

寬度: (2, 303)
高度: (2, 303)
深度: (2, 303)
Task09_Spleen, 1類


100%|██████████| 41/41 [01:37<00:00,  2.37s/it]


寬度: (1, 41)
高度: (1, 41)
深度: (1, 41)
Task10_Colon, 1類


100%|██████████| 126/126 [06:19<00:00,  3.01s/it]

寬度: (1, 126)
高度: (1, 126)
深度: (1, 126)
